In [41]:
import pandas as pd
import numpy as np
import os
import sys
import argparse
import logging
import time
import json
import re
import random
import time

In [42]:
oil_data = pd.read_csv(r'D:\project\Forecasting-CPI-in-Viet-Nam\data\Dữ liệu Lịch sử Hợp Đồng Tương Lai Dầu Thô WTI.csv')
gold_data = pd.read_csv(r'D:\project\Forecasting-CPI-in-Viet-Nam\data\Dữ liệu Lịch sử Hợp đồng Tương lai Vàng.csv')
interest_rate_data = pd.read_csv(r'D:\project\Forecasting-CPI-in-Viet-Nam\data\Dữ liệu Lịch sử Hợp đồng Tương lai Lãi suất Tiền gửi Liên Ngân hàng Một Ngày.csv')

In [43]:
print("Oil Data:")
print(oil_data.head())
print(oil_data.info())
print("Gold Data:")
print(gold_data.head())
print(gold_data.info())
print("Interest Rate Data:")
print(interest_rate_data.head())
print(interest_rate_data.info())

Oil Data:
         Ngày  Lần cuối     Mở    Cao   Thấp       KL % Thay đổi
0  01/05/2025     59.62  58.16  59.86  55.39  887.51K      2.42%
1  01/04/2025     58.21  71.39  72.28  55.12    6.86M    -18.56%
2  01/03/2025     71.48  69.95  71.83  65.22    5.05M      2.47%
3  01/02/2025     69.76  74.14  75.18  68.36    4.90M     -3.82%
4  01/01/2025     72.53  71.85  80.77  71.79    6.29M      1.13%
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Ngày        365 non-null    object 
 1   Lần cuối    365 non-null    float64
 2   Mở          365 non-null    float64
 3   Cao         365 non-null    float64
 4   Thấp        365 non-null    float64
 5   KL          364 non-null    object 
 6   % Thay đổi  365 non-null    object 
dtypes: float64(4), object(3)
memory usage: 20.1+ KB
None
Gold Data:
         Ngày  Lần cuối        Mở       Cao      Thấp     

In [44]:
import pandas as pd
import numpy as np
import os

# Hàm làm sạch giá trị số
def clean_numeric(value):
    try:
        cleaned = str(value).replace(',', '').replace('%', '')
        return float(cleaned)
    except (ValueError, TypeError):
        return np.nan

# Hàm phát hiện outlier bằng IQR
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    df.loc[df[column] > Q3 + 1.5 * IQR, column] = np.nan
    df.loc[df[column] < Q1 - 1.5 * IQR, column] = np.nan
    return df

# Xử lý định dạng và làm sạch dữ liệu
for df in [oil_data, gold_data, interest_rate_data]:
    df['Ngày'] = pd.to_datetime(df['Ngày'], format='%d/%m/%Y')
    if df['Lần cuối'].dtype == 'object':
        df['Lần cuối'] = df['Lần cuối'].apply(clean_numeric)
    df = remove_outliers(df, 'Lần cuối')

# Chuyển Ngày về datetime
for df in [oil_data, gold_data, interest_rate_data]:
    df['Ngày'] = df['Ngày'].astype('period[M]').dt.to_timestamp()

# Chọn cột cần thiết và đổi tên
oil_data = oil_data[['Ngày', 'Lần cuối']].rename(columns={'Lần cuối': 'oil_price'})
gold_data = gold_data[['Ngày', 'Lần cuối']].rename(columns={'Lần cuối': 'gold_price'})
interest_rate_data = interest_rate_data[['Ngày', 'Lần cuối']].rename(columns={'Lần cuối': 'interest_rate'})

# Gộp dữ liệu bằng outer merge
exog_data = oil_data.merge(gold_data, on='Ngày', how='outer').merge(interest_rate_data, on='Ngày', how='outer')

# Sắp xếp theo ngày
exog_data = exog_data.sort_values('Ngày')
# Đặt chỉ số là Ngày
exog_data.set_index('Ngày', inplace=True)
exog_data.index.freq = 'MS'

print("Exogenous Data:")
print(exog_data.head())

Exogenous Data:
            oil_price  gold_price  interest_rate
Ngày                                            
1995-01-01      18.39       377.7            NaN
1995-02-01      18.49       378.3            NaN
1995-03-01      19.17       394.3            NaN
1995-04-01      20.38       388.5            NaN
1995-05-01      18.89       387.6            NaN


In [45]:
# In giá trị thiếu trước nội suy
print("Giá trị thiếu nội suy:")
print(exog_data[['oil_price', 'gold_price', 'interest_rate']].isnull().sum())


Giá trị thiếu nội suy:
oil_price         0
gold_price        2
interest_rate    77
dtype: int64


In [46]:
# Nội suy giá trị thiếu
exog_data[['oil_price', 'gold_price', 'interest_rate']] = exog_data[['oil_price', 'gold_price', 'interest_rate']].interpolate(method='linear')

# Sau đó nội suy tiến/lùi để xử lý NaN đầu/cuối
# exog_data[['oil_price', 'gold_price', 'interest_rate']] = exog_data[['oil_price', 'gold_price', 'interest_rate']].fillna(method='bfill').fillna(method='ffill')

# In thông tin dữ liệu gộp
print("\nDữ liệu ngoại sinh sau khi gộp và chuẩn hóa:")
print(exog_data.head())
print(exog_data.info())
print("Giá trị thiếu sau khi nội suy:")
print(exog_data.isnull().sum())


Dữ liệu ngoại sinh sau khi gộp và chuẩn hóa:
            oil_price  gold_price  interest_rate
Ngày                                            
1995-01-01      18.39       377.7            NaN
1995-02-01      18.49       378.3            NaN
1995-03-01      19.17       394.3            NaN
1995-04-01      20.38       388.5            NaN
1995-05-01      18.89       387.6            NaN
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 365 entries, 1995-01-01 to 2025-05-01
Freq: MS
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   oil_price      365 non-null    float64
 1   gold_price     365 non-null    float64
 2   interest_rate  350 non-null    float64
dtypes: float64(3)
memory usage: 11.4 KB
None
Giá trị thiếu sau khi nội suy:
oil_price         0
gold_price        0
interest_rate    15
dtype: int64


In [47]:
# Nội suy giá trị thiếu
exog_data[['interest_rate']] = exog_data[['interest_rate']].interpolate(method='linear')
# In thông tin dữ liệu gộp
print("\nDữ liệu ngoại sinh sau khi gộp và chuẩn hóa:")
print(exog_data.head())
print(exog_data.info())
print("Giá trị thiếu sau khi nội suy:")
print(exog_data.isnull().sum())


Dữ liệu ngoại sinh sau khi gộp và chuẩn hóa:
            oil_price  gold_price  interest_rate
Ngày                                            
1995-01-01      18.39       377.7            NaN
1995-02-01      18.49       378.3            NaN
1995-03-01      19.17       394.3            NaN
1995-04-01      20.38       388.5            NaN
1995-05-01      18.89       387.6            NaN
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 365 entries, 1995-01-01 to 2025-05-01
Freq: MS
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   oil_price      365 non-null    float64
 1   gold_price     365 non-null    float64
 2   interest_rate  350 non-null    float64
dtypes: float64(3)
memory usage: 11.4 KB
None
Giá trị thiếu sau khi nội suy:
oil_price         0
gold_price        0
interest_rate    15
dtype: int64


In [48]:
# Lưu dữ liệu gộp
exog_data.to_csv('exog_data.csv')

# Kiểm tra dữ liệu bất thường
print("\nKiểm tra giá trị bất thường:")
print("Giá trị âm trong oil_price:", (exog_data['oil_price'] < 0).sum())
print("Giá trị âm trong gold_price:", (exog_data['gold_price'] < 0).sum())
print("Giá trị âm trong interest_rate:", (exog_data['interest_rate'] < 0).sum())



Kiểm tra giá trị bất thường:
Giá trị âm trong oil_price: 0
Giá trị âm trong gold_price: 0
Giá trị âm trong interest_rate: 0


In [53]:
import pandas as pd
import numpy as np
import os

# Load và tiền xử lý dữ liệu
cpi_data = pd.read_csv('cpi.csv', usecols=['t', 'cpi'])
cpi_data.rename(columns={'t': 'date'}, inplace=True)
cpi_data['time'] = pd.to_datetime(cpi_data['date'], format='%b-%y')
cpi_data.set_index('time', inplace=True)
cpi_data['cpi'] = cpi_data['cpi']
cpi_data = cpi_data[['cpi']].dropna()

exog_data = pd.read_csv('exog_data.csv')
exog_data['Ngày'] = pd.to_datetime(exog_data['Ngày'])
exog_data.set_index('Ngày', inplace=True)
exog_cols = ['oil_price', 'gold_price', 'interest_rate']

# Join và xử lý NaN
data = cpi_data.join(exog_data, how='inner')
data.head()

,cpi,oil_price,gold_price,interest_rate
1995-01-01,103.8,18.39,377.7,NaN
1995-02-01,103.4,18.49,378.3,NaN
1995-03-01,100.2,19.17,394.3,NaN
1995-04-01,101.0,20.38,388.5,NaN
1995-05-01,101.8,18.89,387.6,NaN


In [54]:
data = data[['cpi', 'oil_price', 'gold_price']]
data.head()

,cpi,oil_price,gold_price
1995-01-01,103.8,18.39,377.7
1995-02-01,103.4,18.49,378.3
1995-03-01,100.2,19.17,394.3
1995-04-01,101.0,20.38,388.5
1995-05-01,101.8,18.89,387.6


In [55]:
data.to_csv('data.csv')
print("Dữ liệu đã được lưu vào data.csv")

Dữ liệu đã được lưu vào data.csv
